In [34]:
import pandas as pd
import geopandas as gpd
# from geopy.geocoders import Nominatim
from shapely.geometry import Point

In [35]:
# Load in GeoJSON
gdf = gpd.read_file("C:/Users/mford.MSBIOWORKS/Project_1/gz_2010_us_050_00_5m.json")
# print(monthly_energy_production_df)
gdf.head(5)

# us_states = gpd.read_file('us_states.geojson')

TypeError: startswith first arg must be bytes or a tuple of bytes, not str

In [19]:
row_count = len(gdf)
print("Number of rows in the DataFrame:", row_count)

Number of rows in the DataFrame: 52


In [20]:
data_types = gdf.dtypes
print(data_types)

GEO_ID          object
STATE           object
NAME            object
LSAD            object
CENSUSAREA     float64
geometry      geometry
dtype: object


In [21]:
# Call the USWTDB API and apply custom URL parameters to the request. Parameters allow us to filter the data return.
data_url = "https://eersc.usgs.gov/api/uswtdb/v1/turbines?&t_cap=gt.0&select=case_id,xlong,ylat"

# Parse the JSON response from the API return and populate the dataframe
dfClusterMap = pd.read_json(data_url)

# Preview the first five records of our dataframe based on the custom URL paramters in the API request
dfClusterMap.head(5)


,case_id,xlong,ylat
0,3123985,-99.78703,36.50172
1,3123544,-99.72562,36.43713
2,3123887,-99.76972,36.44493
3,3123765,-99.80706,36.51394
4,3123814,-99.75848,36.44498


In [22]:
row_count = len(dfClusterMap)
print("Number of rows in the DataFrame:", row_count)

Number of rows in the DataFrame: 69817


In [23]:
data_types = dfClusterMap.dtypes
print(data_types)

case_id      int64
xlong      float64
ylat       float64
dtype: object


In [24]:
dfClusterMap.rename(columns = {'xlong' : 'long', 'ylat' : 'lat'}, inplace = True)
dfClusterMap.head(5)    

,case_id,long,lat
0,3123985,-99.78703,36.50172
1,3123544,-99.72562,36.43713
2,3123887,-99.76972,36.44493
3,3123765,-99.80706,36.51394
4,3123814,-99.75848,36.44498


In [25]:
# # Assuming you have a GeoDataFrame containing your latitude and longitude data called 'gdf'
# # Perform a spatial join to get the state information
# gdf_with_state = gpd.sjoin(gdf, us_states, how="left", op="within")

# Assuming 'dfClusterMap' is your DataFrame with latitude and longitude columns
# Create a GeoDataFrame with point geometries
geometry = [Point(xy) for xy in zip(dfClusterMap['long'], dfClusterMap['lat'])]


gdfClusterMap = gpd.GeoDataFrame(dfClusterMap, geometry=geometry)

# Set the CRS to EPSG:4326
gdfClusterMap.crs = 'EPSG:4326'

# Now you can perform a spatial join using gdfClusterMap as the GeoDataFrame
result = gpd.sjoin(gdfClusterMap, gdf, how="left", predicate="within")

# 'result' will contain the spatially joined data



In [26]:
result.tail()

,case_id,long,lat,geometry,index_right,GEO_ID,STATE,NAME,LSAD,CENSUSAREA
69812,3003325,-162.55733,66.83433,POINT (-162.55733 66.83433),1.0,0400000US02,02,Alaska,,570640.950
69813,3109704,-116.55776,33.89111,POINT (-116.55776 33.89111),4.0,0400000US06,06,California,,155779.220
69814,3110036,-116.55260,33.89020,POINT (-116.55260 33.89020),4.0,0400000US06,06,California,,155779.220
69815,3110328,-116.55264,33.89168,POINT (-116.55264 33.89168),4.0,0400000US06,06,California,,155779.220
69816,3124485,-83.13453,41.44075,POINT (-83.13453 41.44075),35.0,0400000US39,39,Ohio,,40860.694


In [27]:
row_count = len(result)
print("Number of rows in the DataFrame:", row_count)

Number of rows in the DataFrame: 69817


In [28]:
unique_count = result['NAME'].nunique()
print(unique_count)

43


In [29]:
# Assuming 'result' is your GeoDataFrame
column_headers = result.columns

# Print the column headers
print(column_headers)


Index(['case_id', 'long', 'lat', 'geometry', 'index_right', 'GEO_ID', 'STATE',
       'NAME', 'LSAD', 'CENSUSAREA'],
      dtype='object')


In [30]:
# Assuming 'df' is your DataFrame and 'state_column' is the name of the column with state information
state_counts = result['NAME'].value_counts().reset_index()
state_counts.columns = ['State', 'Count']

# The resulting 'state_counts' DataFrame will contain state names and their corresponding counts
print(state_counts)

             State  Count
0            Texas  18322
1             Iowa   6284
2         Oklahoma   5344
3           Kansas   3927
4       California   3857
5         Illinois   3557
6         Colorado   2825
7        Minnesota   2700
8           Oregon   2113
9     North Dakota   2086
10      New Mexico   1923
11      Washington   1822
12        Michigan   1637
13         Indiana   1602
14        Nebraska   1480
15         Wyoming   1358
16    South Dakota   1327
17        New York   1259
18        Missouri   1081
19         Montana    795
20    Pennsylvania    751
21           Idaho    541
22            Ohio    492
23       Wisconsin    478
24   West Virginia    419
25           Maine    416
26         Arizona    324
27            Utah    205
28          Hawaii    127
29          Alaska    117
30  North Carolina    104
31   Massachusetts     91
32   New Hampshire     84
33        Maryland     80
34         Vermont     73
35          Nevada     66
36     Puerto Rico     60
37    Rhode 

In [31]:
state_counts.to_csv('state_counts.csv', index=False)

In [32]:
result_df = result.drop(columns='geometry')

In [33]:
result_df.head()

,case_id,long,lat,index_right,GEO_ID,STATE,NAME,LSAD,CENSUSAREA
0,3123985,-99.78703,36.50172,36.0,0400000US40,40,Oklahoma,,68594.921
1,3123544,-99.72562,36.43713,36.0,0400000US40,40,Oklahoma,,68594.921
2,3123887,-99.76972,36.44493,36.0,0400000US40,40,Oklahoma,,68594.921
3,3123765,-99.80706,36.51394,36.0,0400000US40,40,Oklahoma,,68594.921
4,3123814,-99.75848,36.44498,36.0,0400000US40,40,Oklahoma,,68594.921
